In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('./data.csv')

data = data[~data['CustomerID'].isna()].reset_index(drop=True)
print len(data)
print data.keys()
import matplotlib.pyplot as plt
data['TotalRevenue'] = data['Quantity']*data['UnitPrice']
print data.iloc[5000:5005]

406829
Index([u'InvoiceNo', u'StockCode', u'Description', u'Quantity', u'InvoiceDate',
       u'UnitPrice', u'CustomerID', u'Country'],
      dtype='object')
     InvoiceNo StockCode                     Description  Quantity  \
5000    537026     22551        PLASTERS IN TIN SPACEBOY        12   
5001    537026    85099B         JUMBO BAG RED RETROSPOT        10   
5002    537026     22355       CHARLOTTE BAG SUKI DESIGN        10   
5003    537026     84992  72 SWEETHEART FAIRY CAKE CASES        24   
5004    537026      POST                         POSTAGE         2   

          InvoiceDate  UnitPrice  CustomerID  Country  TotalRevenue  
5000  12/3/2010 16:35       1.65     12395.0  Belgium          19.8  
5001  12/3/2010 16:35       1.95     12395.0  Belgium          19.5  
5002  12/3/2010 16:35       0.85     12395.0  Belgium           8.5  
5003  12/3/2010 16:35       0.55     12395.0  Belgium          13.2  
5004  12/3/2010 16:35      18.00     12395.0  Belgium          36.0  


In [ ]:
data['Month'] = data['InvoiceDate'].apply(lambda x : x.split('/')[0]).astype(np.uint8)
data['Day'] = data['InvoiceDate'].apply(lambda x : x.split('/')[1]).astype(np.uint8)
data['Year'] = data['InvoiceDate'].apply(lambda x : x.split('/')[2].split(' ')[0]).astype(np.uint16)
data['Time'] = data['InvoiceDate'].apply(lambda x : x.split('/')[2].split(' ')[1])
data['Hour']= data['Time'].apply(lambda x : x.split(':')[0]).astype(np.uint8)
data['Min']= data['Time'].apply(lambda x : x.split(':')[1]).astype(np.uint8)
print data.head()

In [ ]:
temp = data.groupby(['CustomerID',])['TotalRevenue'].apply(sum).reset_index(drop=False)

In [ ]:
#ignoring customers who causes losses because they are small portion of revenue
bins=range(0000,300001,50000)
xx=plt.hist(temp['TotalRevenue'],bins=bins)
print xx
plt.close()
plt.subplots(figsize=(10,10))
xxx=plt.hist(temp['TotalRevenue'],weights = temp['TotalRevenue'].values/1000000, bins=bins)
for i in range(len(xxx[1])-1):
    plt.text(xxx[1][i],xxx[0][i]+0.03,str(int(xx[0][i]))+' Customers',fontsize=12,fontweight='bold')
plt.xticks(np.array(bins)+25000,['Spent\n' + str(bins[x]/1000)+'k-'+str(bins[x+1]/1000)+'k' for x in  range(len(bins)-1)],rotation=60)
plt.xlabel('Spending',fontsize=15,fontweight='bold')
plt.ylabel('Contribution to Revenue in Millions',fontsize=15,fontweight='bold')
plt.show()

In [ ]:
#identiy bad customers

In [ ]:
def f(x):
    if  x[:-1].isdigit() and x[-1].isdigit() is False:
        return x[:-1]
    else : return x
data['StockCode_NR'] = data['StockCode'].apply(f)

#returns cost 0.1%, so ignore first
data = data[data['TotalRevenue']>0].reset_index(drop=True)

In [ ]:
# are there any subgroups of important customer
temp = data.groupby(['CustomerID',])['TotalRevenue'].apply(sum).reset_index(drop=False)
temp2 = temp.sort_values('TotalRevenue').iloc[::-1].reset_index(drop=True)
result,val = [] ,0
for i in range(len(temp2)):
    val += temp2.iloc[i]['TotalRevenue']
    result += [val,]
temp2['cummulative_profit'] = np.array(result)/val
for i in [0.01,0.1,1,2,5,10,20,50]:
    num = int(len(temp2)*1.*i/100)
    print num,
    print 'Top %s percent of customers,%s percent of profit'%(i,100*np.round(temp2.loc[num]['cummulative_profit'],2))
plt.plot(temp2['cummulative_profit']);plt.show()

In [ ]:
# are there any subgroups of important customer
temp = data.groupby(['StockCode_NR'])['TotalRevenue'].apply(sum).reset_index(drop=False)
temp['num_times_brought'] = data.groupby(['StockCode_NR'])['TotalRevenue'].apply(len).reset_index(drop=True)
temp2 = temp.sort_values('num_times_brought').iloc[::-1].reset_index(drop=True)

result,val = [0,] ,0
for i in range(len(temp2)):
    val += temp2.iloc[i]['TotalRevenue']
    result += [val,]
temp2['cummulative_profit'] = np.array(result[1:])/val
temp2['Total_profit'] = (np.array(result[1:])-np.array(result[:-1]))/val
for i in [0.01,0.1,1,2,5,10,20,50,60]:
    num = int(len(temp2)*1.*i/100)
    print num,
    print 'Top %s percent of products,%s percent of profit'%(i,100*np.round(temp2.loc[num]['cummulative_profit'],2))
plt.plot(temp2['cummulative_profit']);plt.show()
x = 10
print np.sum(temp2['num_times_brought']>x)
print 'threshold=',x,',',np.round(np.mean(temp2['num_times_brought']>x),2),'of products account for',
print np.round(np.sum(temp2[temp2['num_times_brought']>x]['TotalRevenue'])/np.sum(temp2['TotalRevenue']),3),'of revenue\n\n'
temp3 = temp2[temp2['num_times_brought']>x]
data2 = pd.merge(data,temp3[['StockCode_NR','num_times_brought','Total_profit']]
         ,on='StockCode_NR',how='inner')


In [ ]:
customer_info = data.groupby(['CustomerID',])['TotalRevenue'].apply(sum).reset_index(drop=False)
customer_info = customer_info.sort_values('TotalRevenue').iloc[::-1].reset_index(drop=False)

for year in pd.unique(data.Year):
    for month in sorted(map(int,pd.unique(data.Month))):
        temp0  =  data2[(data2.Year==year) & (data2.Month==month)]
        if len(temp0) == 0:
            continue
        temp = temp0.groupby(['CustomerID',])['TotalRevenue'].apply(sum).reset_index(drop=False)
        temp2 = temp.sort_values('TotalRevenue').iloc[::-1].reset_index(drop=True)
        result,val = [] ,0
        for i in range(len(temp2)):
            val += temp2.iloc[i]['TotalRevenue']
            result += [val,]
        temp2['cummulative_profit'] = np.array(result)/val
        temp2.rename(columns={'TotalRevenue' : 'Revenue_%s_%s'%(year,month)},inplace=True)
        customer_info = pd.merge(customer_info,temp2[temp2.keys()[:2]],on='CustomerID',how='left').fillna(0)
    

In [ ]:
dictt_StockCode = {}
counter = 0
for i in data2.groupby(['num_times_brought','StockCode_NR']).apply(len).reset_index(drop=False)['StockCode_NR'].iloc[::-1]:
    dictt_StockCode[i] = counter
    counter += 1
data2['StockCode_NR_int'] = data2['StockCode_NR'].map(dictt_StockCode)
print counter

In [ ]:
train = data2[((data2['Year']==2010)*(data2['Month'].isin([12,]))) | \
              ((data2['Year']==2011)*(data2['Month'].isin([1,2,3,4])))]
test = data2[ ((data2['Year']==2011)*(data2['Month'].isin([5])))]
test2 = data2[ ((data2['Year']==2011)*(data2['Month'].isin([6,])))]
test3 = data2[ ((data2['Year']==2011)*(data2['Month'].isin([7,])))]
test4 = data2[ ((data2['Year']==2011)*(data2['Month'].isin([6,7,])))]
test5 = data2[ ((data2['Year']==2011)*(data2['Month'].isin([6,7,8,9,10,11])))]

len(train),len(test),len(test2),len(test3),len(test4),len(test5)

In [ ]:
#get all orders from customer during time period 
Invoice = train.groupby(['CustomerID',]).apply(lambda x :(list(x['StockCode_NR_int']),
                                                         list(x['Quantity']))).reset_index(drop=0)
#print Invoice.head()
customer_info_orders = pd.merge(customer_info,Invoice,on='CustomerID',how='outer')
customer_info_orders.rename(columns={0 : 'orders'},inplace=True)

Invoice_future = test.groupby(['CustomerID',]).apply(lambda x :(list(x['StockCode_NR_int']),
                                                         list(x['Quantity']))).reset_index(drop=0)
#print Invoice.head()
customer_info_orders = pd.merge(customer_info_orders,Invoice_future,on='CustomerID',how='outer')
customer_info_orders.rename(columns={0 : 'orders_future'},inplace=True)

Invoice_future = test2.groupby(['CustomerID',]).apply(lambda x :(list(x['StockCode_NR_int']),
                                                         list(x['Quantity']))).reset_index(drop=0)
#print Invoice.head()
customer_info_orders = pd.merge(customer_info_orders,Invoice_future,on='CustomerID',how='outer')
customer_info_orders.rename(columns={0 : 'orders_future2'},inplace=True)

Invoice_future = test3.groupby(['CustomerID',]).apply(lambda x :(list(x['StockCode_NR_int']),
                                                         list(x['Quantity']))).reset_index(drop=0)
#print Invoice.head()
customer_info_orders = pd.merge(customer_info_orders,Invoice_future,on='CustomerID',how='outer')
customer_info_orders.rename(columns={0 : 'orders_future3'},inplace=True)

Invoice_future = test4.groupby(['CustomerID',]).apply(lambda x :(list(x['StockCode_NR_int']),
                                                         list(x['Quantity']))).reset_index(drop=0)
#print Invoice.head()
customer_info_orders = pd.merge(customer_info_orders,Invoice_future,on='CustomerID',how='outer')
customer_info_orders.rename(columns={0 : 'orders_future4'},inplace=True)

Invoice_future = test5.groupby(['CustomerID',]).apply(lambda x :(list(x['StockCode_NR_int']),
                                                         list(x['Quantity']))).reset_index(drop=0)
#print Invoice.head()
customer_info_orders = pd.merge(customer_info_orders,Invoice_future,on='CustomerID',how='outer')
customer_info_orders.rename(columns={0 : 'orders_future5'},inplace=True)

customer_info_orders = customer_info_orders.sort_values('CustomerID').reset_index(drop=True)
#print customer_info_orders

In [ ]:
matrix_past = np.zeros((len(customer_info_orders),counter)).astype(np.float32)
for i in range(len(customer_info_orders)):
    try:
        pdt,quantity = customer_info_orders.iloc[i]['orders']
        for j,k in zip(pdt,quantity):
            matrix_past[i,j] += k
    except TypeError : None #no orders presetn

import gc 
gc.collect()

In [ ]:
matrix_future = np.zeros((len(customer_info_orders),counter)).astype(np.float32)
for i in range(len(customer_info_orders)):
    try:
        pdt,quantity = customer_info_orders.iloc[i]['orders_future']
        for j,k in zip(pdt,quantity):
            matrix_future[i,j] += k
    except TypeError : None #no orders presetn

import gc 
gc.collect()

In [ ]:
matrix_future2 = np.zeros((len(customer_info_orders),counter)).astype(np.float32)
for i in range(len(customer_info_orders)):
    try:
        pdt,quantity = customer_info_orders.iloc[i]['orders_future2']
        for j,k in zip(pdt,quantity):
            matrix_future2[i,j] += k
    except TypeError : None #no orders presetn

import gc 
gc.collect()

In [ ]:
matrix_future3 = np.zeros((len(customer_info_orders),counter)).astype(np.float32)
for i in range(len(customer_info_orders)):
    try:
        pdt,quantity = customer_info_orders.iloc[i]['orders_future3']
        for j,k in zip(pdt,quantity):
            matrix_future3[i,j] += k
    except TypeError : None #no orders presetn

import gc 
gc.collect()

In [ ]:
matrix_future4 = np.zeros((len(customer_info_orders),counter)).astype(np.float32)
for i in range(len(customer_info_orders)):
    try:
        pdt,quantity = customer_info_orders.iloc[i]['orders_future4']
        for j,k in zip(pdt,quantity):
            matrix_future4[i,j] += k
    except TypeError : None #no orders presetn

import gc 
gc.collect()

In [ ]:
matrix_future5 = np.zeros((len(customer_info_orders),counter)).astype(np.float32)
for i in range(len(customer_info_orders)):
    try:
        pdt,quantity = customer_info_orders.iloc[i]['orders_future5']
        for j,k in zip(pdt,quantity):
            matrix_future5[i,j] += k
    except TypeError : None #no orders presetn

import gc 
gc.collect()

In [ ]:
customer_info_orders.head()

In [ ]:
# recommendation system , for our case since entries are small, we use SVD to do matrix factorization
# instead of a less expensive but approximate algorithm like alternating last squares. WARNING This is not scalable. 
U,D,V = np.linalg.svd(matrix_past)
print len(D)

In [ ]:
num = 500
U2=U[:,:num]
D2=D[:num]
V2=V[:num]
Z = sum(D)
print sum(D2)/Z
plt.plot(map(lambda x : np.sum(D[:x])/Z,range(len(D2))))
plt.show()
print matrix_past
np.matmul(np.matmul(U2,np.diag(D2)),V2)

In [ ]:
customer_info_orders.iloc[0]

In [ ]:
for i in range(counter):
    customer_info_orders[i] = matrix_past[:,i].astype(np.uint16)
for i in range(U2.shape[1]):
    customer_info_orders['svd_'+str(i)] = U2[:,i].astype(np.float32)

In [ ]:
cols

In [ ]:
colsA =  ['Revenue_2011_' + str(x) for x in [1,2,3,4]]+['Revenue_2010_12']
print np.corrcoef(customer_info_orders['target2'].values,
           np.mean(np.log10(1+customer_info_orders[colsA].values),1))

In [ ]:
customer_info_orders.iloc[5]

In [ ]:
customer_info_orders.keys()[3:8]

In [ ]:
import xgboost as xgb
#predict customer orders
'''
train = data2[((data2['Year']==2010)*(data2['Month'].isin([12,]))) | \
              ((data2['Year']==2011)*(data2['Month'].isin([1,2,3,4])))]
test = data2[ ((data2['Year']==2011)*(data2['Month'].isin([5])))]
test2 = data2[ ((data2['Year']==2011)*(data2['Month'].isin([6,])))]
test3 = data2[ ((data2['Year']==2011)*(data2['Month'].isin([7,])))]
test4 = data2[ ((data2['Year']==2011)*(data2['Month'].isin([6,7,])))]
test5 = data2[ ((data2['Year']==2011)*(data2['Month'].isin([6,7,8,9,10,11])))]

len(train),len(test),len(test2),len(test3),len(test4),len(test5)
'''
customer_info_orders.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in customer_info_orders.columns.values]

predictors = list(customer_info_orders.keys()[3:8])+\
                     list([x for x in customer_info_orders.keys() if 'svd' in str(x)])+\
                     range(counter) +['mean_log_rev_train',]
colsA =  ['Revenue_2011_' + str(x) for x in [1,2,3,4]]+['Revenue_2010_12']
customer_info_orders['mean_log_rev_train']= np.mean(np.log10(1+customer_info_orders[colsA].values),1)

target = 'target'
gc.collect()
for i in range(2,6):
    cols = [[0,],[0,],[6,],[7,],[6,7],[6,7,8,9,10,11]][i]
    cols = ['Revenue_2011_'+str(x) for x in cols]
    customer_info_orders[target+str(i)]= np.mean(np.log10(1+customer_info_orders[cols].values),1)
    print cols
print customer_info_orders.keys()
if True:
    customer_info_orders[target]= np.log10(1+np.mean(customer_info_orders[['Revenue_2011_5',]].values,1))
    dtrain = customer_info_orders.iloc[::2]
    dcv = customer_info_orders.iloc[1::2]
    dtest = customer_info_orders
    gc.collect()
    params = {}
    params["objective"] = "reg:linear"
    params["eta"] = 0.05
    params["min_child_weight"] = 1
    params["subsample"] = 0.3
    params["colsample_bytree"] = 0.3
    params["scale_pos_weight"] = 1.0
    params["silent"] = 1
    params["verbose"] = 1
    params["max_depth"] = 5
    #params["nthread"] = 6
    params["nthread"] = -1
    early_stopping_rounds = 50
    plst = list(params.items())
    xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
    xgcv = xgb.DMatrix(dcv[predictors].values, label=dcv[target].values)
    xgtest2 = xgb.DMatrix(dtest[predictors].values, label=dtest[target+'2'].values)
    xgtest3 = xgb.DMatrix(dtest[predictors].values, label=dtest[target+'3'].values)
    xgtest4 = xgb.DMatrix(dtest[predictors].values, label=dtest[target+'4'].values)
    xgtest5 = xgb.DMatrix(dtest[predictors].values, label=dtest[target+'5'].values)
    #    xgb.train(xgtrain[predictors],dtrain['Demanda_uni_equil'],eval_set=evallist ,
    #            eval_metric='rmse', early_stopping_rounds=early_stopping_rounds)
    watchlist  = [ (xgtrain,'train'),(xgtest2,'2month'),(xgtest3,'3month'),
                  (xgtest4,'4month'),(xgtest5,'5month'),(xgcv,'eval')]
    a = {}
    model=xgb.train(plst,xgtrain,4500,watchlist,verbose_eval =5000,
                    early_stopping_rounds=early_stopping_rounds,evals_result=a)
    print np.corrcoef(model.predict(xgtest2)[1::2],customer_info_orders['target'].iloc[1::2])
    for j in range(2,6):
        j = str(j)
        print np.corrcoef(model.predict(xgtest2),customer_info_orders['target'+j])

In [ ]:
plt.hist(customer_info_orders['target5']);plt.show()
plt.hist(customer_info_orders['target4']);plt.show()
plt.hist(customer_info_orders['target3']);plt.show()
plt.hist(customer_info_orders['target2']);plt.show()

plt.plot(model.predict(xgtest3),customer_info_orders['target'],'ro',markersize=1);plt.show()
plt.plot(model.predict(xgtest2),customer_info_orders['target5'],'go',markersize=1);plt.show()
#plt.plot(model.predict(xgtest3),customer_info_orders['target4'],'bo')
#plt.plot(model.predict(xgtest3),customer_info_orders['target5'],'yo')
plt.show()

In [ ]:
import xgboost as xgb
customer_info_orders.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in customer_info_orders.columns.values]

predictors = list(customer_info_orders.keys()[3:6])+\
                     list([x for x in customer_info_orders.keys() if 'svd' in str(x)])+\
                     range(counter)
target = 'target'
for i in range(10):
    customer_info_orders[target]= np.log10(1+matrix_future[:,i])
    customer_info_orders[target+'2']= np.log10(1+matrix_future2[:,i])
    customer_info_orders[target+'3']= np.log10(1+matrix_future3[:,i])
    customer_info_orders[target+'4']= np.log10(1+matrix_future4[:,i])
    customer_info_orders[target+'5']= np.log10(1+matrix_future5[:,i])
    dtrain = customer_info_orders.iloc[::2]
    dcv = customer_info_orders.iloc[1::2]
    dtest = customer_info_orders
    gc.collect()
    params = {}
    params["objective"] = "reg:linear"
    params["eta"] = 0.05
    params["min_child_weight"] = 1
    params["subsample"] = 0.3
    params["colsample_bytree"] = 0.3
    params["scale_pos_weight"] = 1.0
    params["silent"] = 1
    params["verbose"] = 1
    params["max_depth"] = 5
    #params["nthread"] = 6
    params["nthread"] = -1
    early_stopping_rounds = 50
    plst = list(params.items())
    xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
    xgcv = xgb.DMatrix(dcv[predictors].values, label=dcv[target].values)
    xgtest2 = xgb.DMatrix(dtest[predictors].values, label=dtest[target+'2'].values)
    xgtest3 = xgb.DMatrix(dtest[predictors].values, label=dtest[target+'3'].values)
    xgtest4 = xgb.DMatrix(dtest[predictors].values, label=dtest[target+'4'].values)
    xgtest5 = xgb.DMatrix(dtest[predictors].values, label=dtest[target+'5'].values)
    #    xgb.train(xgtrain[predictors],dtrain['Demanda_uni_equil'],eval_set=evallist ,
    #            eval_metric='rmse', early_stopping_rounds=early_stopping_rounds)
    watchlist  = [ (xgtrain,'train'),(xgtest2,'2month'),(xgtest3,'3month'),
                  (xgtest4,'4month'),(xgtest5,'5month'),(xgcv,'eval')]
    a = {}
    model=xgb.train(plst,xgtrain,4500,watchlist,verbose_eval =5000,
                    early_stopping_rounds=early_stopping_rounds,evals_result=a)
    for j in range(2,6):
        j = str(j)
        print np.corrcoef(model.predict(xgtest3),customer_info_orders['target'+j])

In [ ]:
params = {}
params["objective"] = "reg:linear"
params["eta"] = 0.05
params["min_child_weight"] = 1
params["subsample"] = 0.3
params["colsample_bytree"] = 0.3
params["scale_pos_weight"] = 1.0
params["silent"] = 1
params["verbose"] = 1
params["max_depth"] = 5
#params["nthread"] = 6
params["nthread"] = -1
early_stopping_rounds = 50
plst = list(params.items())
xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
xgcv = xgb.DMatrix(dcv[predictors].values, label=dcv[target].values)
xgtest2 = xgb.DMatrix(dtest[predictors].values, label=dtest[target+'2'].values)
xgtest3 = xgb.DMatrix(dtest[predictors].values, label=dtest[target+'3'].values)
xgtest4 = xgb.DMatrix(dtest[predictors].values, label=dtest[target+'4'].values)
xgtest5 = xgb.DMatrix(dtest[predictors].values, label=dtest[target+'5'].values)
#    xgb.train(xgtrain[predictors],dtrain['Demanda_uni_equil'],eval_set=evallist ,
#            eval_metric='rmse', early_stopping_rounds=early_stopping_rounds)
watchlist  = [ (xgtrain,'train'),(xgtest2,'2month'),(xgtest3,'3month'),
              (xgtest4,'4month'),(xgtest5,'5month'),(xgcv,'eval')]
a = {}
model=xgb.train(plst,xgtrain,4500,watchlist,verbose_eval =50,
                early_stopping_rounds=early_stopping_rounds,evals_result=a)
model.get_fscore

In [ ]:
U.shape

In [ ]:
V.shape

In [ ]:

for i in range(len(predictors)):
    try:
        print model.get_fscore()['f'+str(i)],predictors[i]
    except : 
        print 0,predictors[i]

In [ ]:
np.matmul(np.matmul(U,np.diag(np.concatenate([D,[0,]*(len(U)-len(D))]))[:,:len(V)]),V)